# LangChain `@tool` 데코레이터

In [113]:
from datetime import datetime
from dotenv import load_dotenv
import pytz
from isapi.isapicon import HTTP_UNAUTHORIZED
from langchain_core.tools import tool   # decorator
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [114]:
# .env 파일에 저장된 api_key를 OS 환경 변수로 로딩
load_dotenv()

True

In [115]:
# OpenAI 클라이언트를 생성
model = ChatOpenAI(model = 'gpt-4o-mini')

In [116]:
# AI에게 메시지 전달하고 실행(invoke).
messages = [
    SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.'),
    HumanMessage(content='지금 현재 서울 시간?')
]
ai_message = model.invoke(input=messages)

In [117]:
type(ai_message)

langchain_core.messages.ai.AIMessage

In [118]:
print(ai_message)

content='서울의 현재 시간을 확인하려면 인터넷에서 시간을 확인해 보시기 바랍니다. 제가 실시간 정보를 제공할 수는 없지만, 서울은 한국 표준시(KST)로 UTC+9 시간대에 있습니다. 따라서 다른 지역의 시간을 기준으로 변환하시면 서울의 현재 시간을 알 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 30, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CGGHGSjMECvY3kTnuuLNCTHoI3qGd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e3a79987-4ef1-41e1-8e4f-0529b411078d-0' usage_metadata={'input_tokens': 30, 'output_tokens': 65, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [119]:
ai_message.pretty_print()

================================== Ai Message ==================================

서울의 현재 시간을 확인하려면 인터넷에서 시간을 확인해 보시기 바랍니다. 제가 실시간 정보를 제공할 수는 없지만, 서울은 한국 표준시(KST)로 UTC+9 시간대에 있습니다. 따라서 다른 지역의 시간을 기준으로 변환하시면 서울의 현재 시간을 알 수 있습니다.


도구(tool)을 AI에게 제공하고, AI는 tool 목록에 있는 함수 호출(function calling)을 요청해서 에이전트가 함수 호출 결과를 다시 AI에게 전송하면 AI는 함수 호출 결과를 바탕으로 답변을 생성할 수 있음.

사용자 질문(도구 목록 제공) -> AI 도구 호출 요청 -> 사용자 함수 호출 결과 제공 -> AI 답변 생성

In [120]:
@tool
def get_current_time(timezone: str, location: str) -> str:
    """해당 timezone의 현재 날짜와 시간을 문자열로 리턴.

    Args:
        timezone (str) : 타임존. (예: Asia/Seoul)
        location (str) : 지역명. 타임존은 모든 도시 이름에 대응되지 않기 때문에 LLM이 답변을 생성할 때 이용하도록 제공.
    Returns: '날짜 시간 타임존(지역명)' 형식의 문자열을 반환. (예: 2025-09-15 15:30:55 Asia/Seoul(부산))
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
    result = f'{now} {timezone}({location})'

    return result

In [121]:
# 도구 목록 : @tool 데코레이터가 사용된 함수들의 리스트
tools = [get_current_time,]

In [122]:
# AI에서 도구 목록의 함수 호출을 요청했을 때 함수 객체를 쉽게 찾기 위해서 dict 선언.
tool_dict = {
    'get_current_time': get_current_time,
}

In [123]:
# AI 모델과 도구 목록을 binding (묶어줌).
model_with_tools = model.bind_tools(tools=tools)

In [124]:
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답하는 AI 비서야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='지금 현재 서울 시간?', additional_kwargs={}, response_metadata={})]


In [125]:
# 도구 목록을 가지고 있는 AI 모델 호출
response = model_with_tools.invoke(input=messages)

In [126]:
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


In [127]:
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_249lctUKDSXYg8rDlJ5LGssh', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 170, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGGHIy3T9qRVMGLDbQAfCLimAwWS3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b9023ffc-c5ed-4986-8393-ebf11e19befb-0' tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '서울'}, 'id': 'call_249lctUKDSXYg8rDlJ5LGssh', 'type': 'tool_call'}] usage_metadata={'input_tokens': 170, 'output_tokens': 22, '

In [128]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_249lctUKDSXYg8rDlJ5LGssh)
 Call ID: call_249lctUKDSXYg8rDlJ5LGssh
  Args:
    timezone: Asia/Seoul
    location: 서울


In [129]:
messages.append(response)   # 대화 이력 저장 : AIMessages 객체를 리스트에 추가

In [130]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_249lctUKDSXYg8rDlJ5LGssh)
 Call ID: call_249lctUKDSXYg8rDlJ5LGssh
  Args:
    timezone: Asia/Seoul
    location: 서울


In [131]:
for tool_call in response.tool_calls:
    fn = tool_dict[tool_call['name']]   # AIMessage의 포함된 함수 이름으로 함수 객체를 찾음.
    tool_msg = fn.invoke(tool_call)     # @tool 데코레이터로 포장 -> invoke가 가능.
    # invoke : tool_call에서 args를 찾고 fn를 호출할 때 아규먼트를 전달. 리턴값을 이용해서 ToolMessage 객체 반환.
    messages.append(tool_msg)

In [132]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답하는 AI 비서야.
================================ Human Message =================================

지금 현재 서울 시간?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_249lctUKDSXYg8rDlJ5LGssh)
 Call ID: call_249lctUKDSXYg8rDlJ5LGssh
  Args:
    timezone: Asia/Seoul
    location: 서울
================================= Tool Message =================================
Name: get_current_time

2025-09-16 12:04:17 Asia/Seoul(서울)


In [133]:
ai_message = model_with_tools.invoke(input=messages)

In [134]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 서울 시간은 2025-09-16 12:04:17입니다.


# 2개 이상의 도구 호출을 포함하는 tool_calls 리스트

In [135]:
messages = [
    SystemMessage('사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.'),
    SystemMessage('도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.'),
    HumanMessage('서울, LA, 런던의 현재 시간은?'),
]

In [136]:
ai_message = model_with_tools.invoke(input=messages)

In [137]:
messages.append(ai_message)

In [138]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, LA, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_nK3wXiob5AdNRrRWzlcqswqX)
 Call ID: call_nK3wXiob5AdNRrRWzlcqswqX
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_WztA2jJOEeVsJvvGja6Jk0Fu)
 Call ID: call_WztA2jJOEeVsJvvGja6Jk0Fu
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_RSVvYlDukaE7A6wWMcwNPuM8)
 Call ID: call_RSVvYlDukaE7A6wWMcwNPuM8
  Args:
    timezone: Europe/London
    location: 런던


In [139]:
for tool_call in ai_message.tool_calls:
    fn = tool_dict[tool_call['name']]
    tool_msg = fn.invoke(tool_call)
    messages.append(tool_msg)

In [140]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

사용자의 질문에 답하기 위해서 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있으니, 정확한 타임존을 선택해줘.
================================ Human Message =================================

서울, LA, 런던의 현재 시간은?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_nK3wXiob5AdNRrRWzlcqswqX)
 Call ID: call_nK3wXiob5AdNRrRWzlcqswqX
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_WztA2jJOEeVsJvvGja6Jk0Fu)
 Call ID: call_WztA2jJOEeVsJvvGja6Jk0Fu
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_RSVvYlDukaE7A6wWMcwNPuM8)
 Call ID: call_RSVvYlDukaE7A6wWMcwNPuM8
  Args:
    timezone: Europe/London
    location: 런던
================================= Tool Message =================================
Name: get_current_time

2025-09-16 12:04:20 Asia/Se

In [141]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_nK3wXiob5AdNRrRWzlcqswqX)
 Call ID: call_nK3wXiob5AdNRrRWzlcqswqX
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_WztA2jJOEeVsJvvGja6Jk0Fu)
 Call ID: call_WztA2jJOEeVsJvvGja6Jk0Fu
  Args:
    timezone: America/Los_Angeles
    location: LA
  get_current_time (call_RSVvYlDukaE7A6wWMcwNPuM8)
 Call ID: call_RSVvYlDukaE7A6wWMcwNPuM8
  Args:
    timezone: Europe/London
    location: 런던


# Pydantic

입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확히 표현하는 라이브러리.

In [142]:
def subtract(x: int, y: int) -> int:
    return x - y

In [143]:
subtract(1, 2)

-1

In [144]:
subtract(1.1, 2.5)

-1.4

Python은 파라미터의 타입 검사를 하지 않고, 리턴 타입도 힌트대로 반환하는 것은 아님.

In [145]:
class Person:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name

In [146]:
person = Person(id=1, name='오쌤')

In [147]:
print(person.id)

1


In [148]:
person2 = Person(id='abc123', name=123)

In [149]:
print(person2.id)

abc123


Python은 객체를 생성할 때 속성(property)들의 타입 검사를 수행하지 않음.

In [150]:
from pydantic import BaseModel, ValidationError, PositiveInt

In [151]:
class Person(BaseModel):    # 부모 클래스 상속, Person 클래스는 BaseModel 클래스를 상속(확장).
    id: PositiveInt
    name: str

In [152]:
person3 = Person(id=12345, name='홍길동')

In [153]:
print(person3)

id=12345 name='홍길동'


In [154]:
person3.model_dump()    # 인스턴스의 속성(property) 이름과 값으로 dict를 생성해서 리턴.

{'id': 12345, 'name': '홍길동'}

In [155]:
try:
    person4 = Person(id='abc123', name='1_000')
    print(person4)
except ValidationError as e:
    print(e)

1 validation error for Person
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [156]:
person5 = Person(**{'id' : 123456, 'name' : '길동'})  # ** : unpacking 연산자
print(person5)

id=123456 name='길동'


# Pydandtic을 이용한 함수 파라미터 타입 클래스 선언

In [180]:
from pydantic import Field

In [181]:
# 주식 가격 조회 함수의 파라미터 타입으로 사용할 클래스.
class StockHistoryArgs(BaseModel):
    ticker: str = Field(..., title='주식 코드',
                        description='주식 데이터를 검색하기 위한 주식 코드(예: AAPL, AMZN)')
    period: str = Field(..., title='기간',
                        description='주식 데이터 조회 기간(예: 1d, 1mo, 1y)')

In [182]:
StockHistoryArgs.model_json_schema()

{'properties': {'ticker': {'description': '주식 데이터를 검색하기 위한 주식 코드(예: AAPL, AMZN)',
   'title': '주식 코드',
   'type': 'string'},
  'period': {'description': '주식 데이터 조회 기간(예: 1d, 1mo, 1y)',
   'title': '기간',
   'type': 'string'}},
 'required': ['ticker', 'period'],
 'title': 'StockHistoryArgs',
 'type': 'object'}

In [183]:
import yfinance as yf

In [184]:
@tool
def get_yf_stock_history(input: StockHistoryArgs):
    """해당 기간(period) 동안 주식 코드(ticker)의 데이터를 조회하는 함수. """
    ticker = yf.Ticker(input.ticker)
    history = ticker.history(period=input.period)
    return history.to_markdown()    # 조회한 데이터를 마크다운(Markdown) 형식의 문자열로 리턴.

In [188]:
tool_dict ={
    'get_current_time': get_current_time,
    'get_yf_stock_history': get_yf_stock_history,
}

In [189]:
model_with_tools = model.bind_tools(tools=tools)    # 도구 목록을 포함하는 AI 모델 객체

In [190]:
messages = [
    SystemMessage('답변을 생성하기 위해 도구를 사용할 수 있어.'),
    SystemMessage('도시 이름과 타임존은 일치하지 않을 수도 있어.'),
    SystemMessage('ticker는 실제 사용되는 주식 코드여야 해.'),
    HumanMessage('삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?'),
]

In [191]:
ai_message = model_with_tools.invoke(input=messages)

In [192]:
messages.append(ai_message)

In [193]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

답변을 생성하기 위해 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있어.
================================ System Message ================================

ticker는 실제 사용되는 주식 코드여야 해.
================================ Human Message =================================

삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_mBqXEMSxHJd42eGYW8UuNSv9)
 Call ID: call_mBqXEMSxHJd42eGYW8UuNSv9
  Args:
    timezone: Asia/Seoul
    location: 서울


In [176]:
for tool_call in ai_message.tool_calls:
    fn = tool_dict[tool_call['name']]
    tool_msg = fn.invoke(tool_call)
    messages.append(tool_msg)

In [177]:
for m in messages:
    m.pretty_print()

================================ System Message ================================

답변을 생성하기 위해 도구를 사용할 수 있어.
================================ System Message ================================

도시 이름과 타임존은 일치하지 않을 수도 있어.
================================ System Message ================================

ticker는 실제 사용되는 주식 코드여야 해.
================================ Human Message =================================

삼성전자는 한 달 전과 비교해서 주식이 올랐어? 내렸어?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_8wpijb4Bl7uMreCddpQl2Leo)
 Call ID: call_8wpijb4Bl7uMreCddpQl2Leo
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_mLyFx7sSAPUZZN1TZ1ZlkMMV)
 Call ID: call_mLyFx7sSAPUZZN1TZ1ZlkMMV
  Args:
    timezone: America/New_York
    location: 뉴욕
================================= Tool Message =================================
Name: get_current_time

2025-09-16 12:50:53 Asia/Seoul(서울)
================================= Tool 

In [178]:
ai_message = model_with_tools.invoke(input=messages)

In [179]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 날짜는 2025년 9월 16일입니다.

삼성전자의 주가 변동을 확인하기 위해 한 달 전인 8월 16일의 주가와 현재 주가를 비교해야 합니다. 그러나 현재 삼성전자의 주가 정보는 확인할 수 없습니다. 주가를 확인하려면 주식 정보 제공 웹사이트나 금융 서비스에서 직접 찾아보셔야 합니다.
